# Azure OpenAI Function Calling을 카카오 모빌리티 API와 통합 예시
이 파일은 Kakao Mobility의 실시간 검색 정보를 활용할 수 있는 GPT 모델 사용 예시입니다.

참고: 이 예시는 외부 API를 호출하도록 만들어져 있습니다. 따라서 [KAKAO API](https://developers.kakaomobility.com/docs/navi-api/directions/)에서 제공하는 `KEY를 발급받아 .env 파일에 등록`하고 사용하여야 합니다.

## Setup

In [1]:
import openai
import json
import os
import requests
import pytz
from urllib import parse
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

# Azure OpenAI resource 정보를 설정합니다.
openai.api_type     = os.getenv("OPENAI_API_TYPE")
openai.api_key      = os.getenv("OPENAI_API_KEY")
openai.api_base     = os.getenv("OPENAI_API_BASE")
openai.api_version  = os.getenv("OPENAI_API_VERSION")   # API 버전은 "2023-07-01-preview" 부터 사용 가능합니다.
deployment_id       = os.getenv("DEPLOYMENT_NAME_16K")  # Azure OpenAI resource의 deployment id를 입력합니다.
KAKAO_API_KEY       = os.getenv("KAKAO_REST_API_KEY")   # KAKAO REST API 키입니다.

## 1. Kakao API functions 정의
이제 함수로 작업하는 방법을 알았으므로 코드에서 몇 가지 함수를 정의하여 함수를 사용하는 프로세스를 끝까지 살펴보겠습니다.

In [2]:
headers = {
    "Authorization": "KakaoAK " + KAKAO_API_KEY,
    "Content-Type": "application/json",
}    

# Kakao 키워드 기반 위경도 좌표 찾기
def get_location_xy(keyword="판교 알파돔타워"):
    params = {
        "query": keyword
    }
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?" + parse.urlencode(params)
    response = requests.get(url, headers=headers)
    return (response.json()["documents"][0]["x"] + "," + response.json()["documents"][0]["y"] + ",name=" + response.json()["documents"][0]["place_name"])

def get_current_time():
    try:
        # Get the timezone for the city
        timezone = pytz.timezone("Asia/Seoul")

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y%m%d%H%M")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."    

### `Function #1`: 카카오 모빌리티 길찾기 API

카카오 길찾기 API를 통해서 출발지와 목적지 사이의 정보를 탐색합니다.

In [3]:
# Kakao 길찾기 API
def get_directions(origin, destination, waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    origin_xy_info = get_location_xy(origin)
    destin_xy_info = get_location_xy(destination)
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
    }
    url = "https://apis-navi.kakaomobility.com/v1/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": response_summary["distance"],
        "duration": response_summary["duration"],
    }
    
    return json.dumps(return_data)


### `Function #2`: 카카오 모빌리티 미래 길찾기 API

지정된 날짜와 시간으로 카카오 길찾기 API를 통해서 출발지와 목적지 사이의 정보를 탐색합니다.

In [4]:
# Kakao 미래 길찾기 API
def get_future_directions(origin, destination, departure_time=get_current_time(), waypoints="", priority="RECOMMEND", car_fuel="GASOLINE", car_hipass="true", alternatives="false", road_details="false"):
    # 키워드 기반 위경도 좌표 정보 수집
    origin_xy_info = get_location_xy(origin)
    destin_xy_info = get_location_xy(destination)
    
    params = {
        "origin": origin_xy_info,
        "destination": destin_xy_info,
        "waypoints": waypoints,
        "priority": priority,
        "car_fuel": car_fuel,
        "car_hipass": car_hipass,
        "alternatives": alternatives,
        "road_details": road_details,
        "departure_time": departure_time,
    }
    url = "https://apis-navi.kakaomobility.com/v1/future/directions?{}".format("&".join([f"{k}={v}" for k, v in params.items()]))
    response = requests.get(url, headers=headers)
    
    response_summary = response.json()["routes"][0]["summary"]
    return_data = {
        "origin_name": response_summary["origin"]["name"],
        "destination_name": response_summary["destination"]["name"],
        "taxi_fare": response_summary["fare"]["taxi"],
        "toll_fare": response_summary["fare"]["toll"],
        "distance": response_summary["distance"],
        "duration": response_summary["duration"],
    }
    
    return json.dumps(return_data)


### `Function #3`: 시간 포맷 변환

시간 포맷 변환

In [5]:
# caculate from 초 to 시 분 초
def time_calculator(seconds):
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    
    if hours == 0:
        return "%02d분 %02d초" % (minutes, seconds)
    elif minutes == 0:
        return "%02d초" % (seconds)
    else:
        return "%02d시간 %02d분 %02d초" % (hours, minutes, seconds)

## 2. GPT를 사용한 `Function` 호출

`Function Calling`을 위한 단계: 

1. 사용자 쿼리와 functions 매개변수(parameter)에 정의된 함수 집합을 사용하여 모델을 호출합니다.
2. 모델은 함수 호출을 선택할 수 있습니다. 콘텐츠는 사용자 지정 스키마를 준수하는 문자열화된 JSON 객체가 됩니다(참고: 모델이 잘못된 JSON 또는 환각(hallucination) 매개변수를 생성할 수 있음).
3. 코드에서 문자열을 JSON으로 구문 분석합니다. 제공된 인수가 있는 경우 해당 인수로 함수를 호출합니다.
4. 함수 응답을 새 메시지로 추가하여 모델을 다시 호출하고 모델이 결과를 사용자에게 다시 요약하도록 합니다.

### 2.1 모델이 호출 방법을 알 수 있도록 함수 설명

In [6]:

functions = [
        {
            "name": "get_directions",
            "description": "API to search routes based on origin and destination information",
            # "description": "출발지와 도착지 정보를 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},                    
                },
                "required": ["origin", "destination"],
            },
        },
        {
            "name": "get_future_directions",
            "description": "API to search routes based on origin and destination information based on future departure_time",
            # "description": "출발지와 도착지 정보를 미래 시간 기반으로 경로 검색하는 API",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string"},
                    "destination": {"type": "string"},
                    "departure_time": {"type": "string"},
                },
                "required": ["origin", "destination", "departure_time"],
            },
        },
        {
            "name": "time_calculator",
            "description": "Convert seconds to hours minutes seconds",
            # "description": "초를 시분초 단위로 변환",
            "parameters": {
                "type": "object",
                "properties": {
                    "duration": {"type": "integer"},
                },
                "required": ["duration"],
            },
        }
    ]

available_functions = {
            "get_directions": get_directions,
            "get_future_directions": get_future_directions,
            "time_calculator": time_calculator
        } 

### 2.2 function call을 검증하는 helper 함수 정의
모델이 잘못된 function call을 생성할 수 있으므로 function의 유효성을 검사하는 것이 중요합니다. 여기서는 사용 사례에 대해 더 복잡한 유효성 검사를 적용할 수 있지만 함수 호출의 유효성을 검사하는 간단한 helper 함수를 정의합니다.

In [7]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [8]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        # print("Messages in second request:")
        # for message in messages:
        #     print(message)
        # print()
        # print(json.dumps(messages, ensure_ascii=False, indent=4))
        # print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

## 3. 경로 검색 요청

In [9]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are a navigation bot agent. Your reply must absolutely in JSON format. You must never modify or tamper with data."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 알려줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\\\"", "\"")
print(content)

"{
"origin": "판교 알파돔타워",
"destination": "한국마이크로소프트",
"taxi_fare": 33600,
"toll_fare": 0,
"distance": 28183,
"duration": 4570
}"


In [10]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are a bot that guides you through car routes. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. \
    If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 목록을 만들어서 알려줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\"", "")
print(content)

요청하신 판교 알파돔타워에서 한국마이크로소프트까지의 경로 정보는 다음과 같습니다:

- 출발지: 판교 알파돔타워
- 도착지: 한국마이크로소프트
- 택시 요금: 33,600원
- 톨비: 없음
- 총 이동 거리: 28,183m (약 28.18km)
- 총 소요 시간: 4,569초 (약 1시간 16분 9초)

위 정보를 참고하시어 원하시는 경로로 안전하게 이동하시길 바랍니다.


In [11]:
origin_name = "판교 알파돔타워"
destin_name = "한국마이크로소프트"

messages = [
    {"role": "system", "content": "You are car route navigation. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. Summalize sentance without list. \
    If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
    {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 경로 검색한 정보를 문장으로 요약해줘."}
    ]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
content = json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
content = content.replace("\\n", "\n").replace("\"", "")
print(content)

판교 알파돔타워에서 한국마이크로소프트까지의 경로 검색 결과는 다음과 같습니다. 총 거리는 약 28.2km이며, 예상 소요 시간은 약 1시간 16분입니다. 택시 요금은 33,600원이며 톨게이트 요금은 없습니다.


## 4. 미래 운행 정보 기반의 검색

미래 운행 정보 기반의 검색 API를 자동 호출하도록 출발시간을 Prompt에 제공합니다.

In [13]:
origin_name = "한국마이크로소프트"
destin_name = "판교 알파돔타워"

messages = [
        {"role": "system", "content": "You are car route navigation. When the user provides the origin and destination name, you provides summary route guidance information. You should use factual information and not generate text arbitrarily. Summalize sentance without list. \
        If the distance or duration units are too granular, convert them to higher units. For example, 1894 seconds is displayed as 31 minutes and 34 seconds. 4012 seconds is displayed as 1 hour 3 minutes 32 seconds. 12343m should be marked as 12.34km, 145127m should be marked as 124.13km."},
        {"role": "user", "content": f"{origin_name}에서 {destin_name}까지 출발시간이 2023년 8월 1일 11시 0분(%Y%m%d%H%M) 일때, 경로 검색한 정보를 문장으로 요약해줘."}
    ]
assistant_response1 = run_conversation(messages, functions, available_functions, deployment_id)
future_content = json.dumps(assistant_response1['choices'][0]['message']["content"], ensure_ascii=False, indent=4)
future_content = future_content.replace("\\n", "\n").replace("\"", "")
print(future_content)

2023년 8월 1일 11시 0분에 한국마이크로소프트에서 판교 알파돔타워까지의 경로를 검색한 결과는 다음과 같습니다. 출발지는 한국마이크로소프트이고 목적지는 판교 알파돔타워입니다. 총 거리는 약 25.19km이며 총 소요시간은 약 42분 35초입니다. 택시비용은 27,100원이며 통행료는 3,000원입니다.
